### Get Data from TM1

In [ ]:
from TM1py.Services import TM1Service
from TM1py.Utils.Utils import element_names_from_element_unique_names, build_pandas_dataframe_from_cellset
from creds import tm1_credentials

In [ ]:
with TM1Service(address=tm1_credentials["address"], port=tm1_credentials["port"], user=tm1_credentials["user"], password=tm1_credentials["password"], ssl=tm1_credentials["ssl"]) as tm1:
    data = tm1.cubes.cells.execute_view(cube_name='OPS Sales', view_name='Conference View', private=False)
    df = build_pandas_dataframe_from_cellset(data, multiindex=False)

In [ ]:
sales = df.drop(['Version', 'Account','OPS Sales Source', 'OPS Sales Measure'], axis =1 )

### Load Ancillary Data

In [ ]:
import pandas as pd

In [ ]:
def load_data():
    import types
    import pandas as pd

    product = pd.read_csv('Product Details.csv')

    store = pd.read_csv('Store Affluence.csv')

    weather = pd.read_csv('Weather.csv')

    return sales, product, store, weather

In [ ]:
sales, product, store, weather = load_data()

### Prepare Data

In [ ]:
sales['Product'] = sales['Product'].astype(str)
sales['State'] = sales['Store'].apply(lambda x: x[:3])
product['All Products'] = product['All Products'].astype(str)

In [ ]:
import warnings
warnings.filterwarnings("ignore")

weathernsw = weather[['Date', 'NSW Temp Avg', 'NSW Precipitation']]
weathernsw['State'] = 'NSW'
weathernsw.columns = ['Date', 'Temp', 'Preciptation', 'State']

weathervic = weather[['Date', 'VIC Temp Avg', 'VIC Precipitation']]
weathervic['State'] = 'VIC'
weathervic.columns = ['Date', 'Temp', 'Preciptation', 'State']

weatherqld = weather[['Date', 'QLD Temp Avg', 'QLD Precipitation']]
weatherqld['State'] = 'QLD'
weatherqld.columns = ['Date', 'Temp', 'Preciptation', 'State']

weather_table =pd.concat([weathernsw, weathervic, weatherqld])
weather_table['Date'] = weather_table['Date'].apply(lambda x: '0'+str(x) if len(str(x)) < 8 else x)

In [ ]:
df1 = pd.merge(sales,
                 store[['Store', 'RegionAffluence']],
                 left_on='Store', right_on='Store', 
                 how='left')

df2 = pd.merge(df1,
                 product[['Brand Affinity', 'Product Size', 'All Products']],
                 left_on='Product', right_on='All Products', 
                 how='left')
df2.drop(['All Products'], axis=1, inplace=True)

df3 = pd.merge(df2,
                 weather_table,
                 left_on=['State', 'Time Date'], right_on=['State', 'Date'], 
                 how='left')
df3.drop('Date', axis=1, inplace=True)

In [ ]:
df3['Time Date'] = df3['Time Date'].apply(lambda x: '0' + str(x) if len(str(x)) < 8 else x)
df3['Time Date'] = pd.to_datetime(df3['Time Date'], format='%d%m%Y')

In [ ]:
def create_shaped_data(dataframe, store, product, series_length):
    singleprod = dataframe[(dataframe['Store']==store)&(dataframe['Product']==product)]
    singleprod.drop(['Product', 'Store', 'State'], axis=1, inplace=True)
    singleprod['Temp'].fillna(singleprod['Temp'].mean(), inplace=True)
    singleprod['Preciptation'].fillna(singleprod['Preciptation'].mean(), inplace=True)
    
    input_length = series_length
    data_point = 365 - input_length
    abt = []
    timedate = []
    for row in range(input_length, data_point):
        start = row-input_length
        abt.append([singleprod.sort_values(by='Time Date').iloc[start:row,1:].to_numpy()])
        timedate.append([singleprod.sort_values(by='Time Date').iloc[start:row,0].to_numpy()])
        

    abt = np.array(abt)
    abt = abt.reshape(-1,series_length,6)
    
    timedate = np.array(timedate)
    timedate = timedate.reshape(-1,series_length,1)

    X = np.array([steps[:series_length-1] for steps in abt])
    y = np.array([steps[series_length-1][0] for steps in abt])
    y_time = np.array([steps[series_length-1][0] for steps in timedate])
    
    return X, y, y_time

### Start Scoring

In [ ]:
from keras import backend as K
K.clear_session()

In [ ]:
import itertools
import numpy as np
import pickle

In [ ]:
cellset = {}
for product, store in itertools.product(df3['Product'].unique(), df3['Store'].unique()):
    X, y, y_time  = create_shaped_data(df3, store, product, 6)
    K.clear_session()
    modelpath = './models/'+store+'-'+product+'.pkl'
    with open(modelpath, 'rb') as f:
        model = pickle.load(f)
    predictions = model.predict(X.reshape(-1,5,6))
    cellset.update({(pd.to_datetime(str(y_time[count])).strftime('%d%m%Y'), 'LSTM Forecast', '4999', product, store, 'Base', 'Amount'):prediction[0] for count, prediction in enumerate(predictions)})

### Send Data Back into TM1

In [ ]:
with TM1Service(address=tm1_credentials["address"], port=tm1_credentials["port"], user=tm1_credentials["user"], password=tm1_credentials["password"], ssl=tm1_credentials["ssl"]) as tm1:
    tm1.cubes.cells.write_values('OPS Sales', cellset, dimensions=['Time Date', 'Version', 'Account', 'Product', 'Store', 'OPS Sales Source', 'OPS Sales Measure'])

In [ ]:
cellset